In [2]:
import pandas as pd
import glob
import nltk

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# import spacy
from textblob import TextBlob


import requests
from datetime import timedelta, date
from bs4 import BeautifulSoup


In [3]:
def options(option,data,types):
    if(option=="vader"):
        vaderAnalysis(data,types)
        
    if(option=="nltk"):
        nltkClassifier(data,types)
        
    if(option=="both"):
        vaderAnalysis(data,types)
#         print("\n")
        nltkClassifier(data,types)

In [4]:
def vaderAnalysis(data,types):
    if(data!="none"):
        from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

        analyzer = SentimentIntensityAnalyzer()
        pos = 0
        neg = 0
        neu = 0

        vs = analyzer.polarity_scores(data)
        pos = pos+vs['pos']
        neg = neg+vs['neg']
        neu = neu+vs['neu']

        polarity = round(pos-neg,3)
        if(pos>neg):
            print("VADER -",types,", Positive News = {}, Neutral News = {}".format(polarity,neu))
        else:
            print("VADER -",types,", Negative News = {}, Neutral News = {}".format(polarity,neu))
#     print("Positivity - {}, Negativity - {}, Polarity - {}".format(pos, neg,polarity))
#     print("VADER -",types,", = {}".format(polarity))


In [5]:
def nltkClassifier(data,types):
    
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    sid = SentimentIntensityAnalyzer()
    pos_word_list = []

    neu_word_list = []
    neg_word_list = []

    overall = 0
    l=data
    blob = TextBlob(l)
    l = l.lower()
    for word in l.split():
#         print(sid.polarity_scores(word)['compound'], word)
        overall = overall + sid.polarity_scores(word)['compound']
        if (sid.polarity_scores(word)['compound']) >= 0.02:
            pos_word_list.append(word)
        elif (sid.polarity_scores(word)['compound']) <= -0.02:
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)
    overall = round(overall,3)
    if(overall < -0.1):
        print("NLTK  -",types,", Negative news = ", overall)
    elif(overall > 0.1):
        print("NLTK  -",types,", Positive news = ", overall)
    else:
        print("NLTK  -",types,", Neutral  news = ", overall)

In [6]:
def downloadUrl(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    title = soup.find('h1', {"id": "page-title"}).get_text()

    contents = soup.find_all('div', {"class": "content"})
    data = ""
    for content in contents:
        data = data + content.get_text().replace("\n", "").replace("\"", "")

    return (title,data)

In [7]:

def downloadMtlUrl(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    try:
        title = soup.find('h1', {"class": "artTitle"}).get_text()
        data = ""

        sp = soup.find('div', {"id": "article-main"})

        for p in sp.findAll('p'):
            o = p.get_text()
            if(not o.startswith("Disclaimer")):
                data = data + o.replace("\n", "").replace("\"", "")
        return(title, data)
    
    except:
        return("none","none")

In [8]:
def downloadMtlAll(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    i = 0
    b = []
    a = []
    for title in soup.findAll('h2'):
        b.append('')
        a.append('')
        b[i] = title.get_text()
        for z in title.findAll('a'):
            a[i] = z.get('href')
        i = i+1

    return(a)


In [9]:
from datetime import timedelta, date
import re
import requests
url = "https://www.deccanherald.com/getarchive"


def normalizeSpaces(input):
    return input.strip().replace(" ", "-").lower()


def normalizeSentence(input):
    return re.sub(r'[\'\,\:\/\\\(\)]', '', input.strip().lower())

def printLinks(currentDate):
    outs=[]
    response = requests.post(url, json={'arcDate': currentDate})
    soup = BeautifulSoup(response.content, "html.parser")

    headlines = soup.find_all('h2')
    contents = soup.find_all('ul', {'class': 'group'})

    for idx, content in enumerate(contents):
        contentLinks = content.findChildren('li')
        for contentLink in contentLinks:
            hL = headlines[idx].string
            if(hL.startswith("Business")):
                outs.append("https://www.deccanherald.com"+str(contentLink.a['href']))
                
    return outs

urls = printLinks("2020/02/11")
print("Done")

Done


In [10]:
# urls = ["https://www.deccanherald.com/business/amazon-india-inks-long-term-deals-with-future-retail-791766.html","https://www.deccanherald.com/business/business-news/sensex-tanks-788-pts-nifty-falls-235-pts-791748.html","https://www.deccanherald.com/business/business-news/india-incs-overseas-borrowing-up-65-in-nov-2019-791533.html"]
# urls=["https://www.moneycontrol.com/news/business/tech-mahindra-q3-pat-may-dip-10-3-qoq-to-rs-1008-cr-motilal-oswal-4792611.html"]


#For deccan herald


#For moneycontrol
# urls=downloadMtlAll("https://www.moneycontrol.com/news/news-all/page-1/")
urls=downloadMtlAll("https://www.moneycontrol.com/news/business/markets/")

for url in urls:
    if(url!=''):
        title,data = downloadMtlUrl(url)
#         title,data = downloadUrl(url)
        print("\n------\nHeadline - ", title,"\n")
#         options("both",title,"Headline")
#         print("\n=\n")
        options("both",data,"Full")
        print(url)


------
Headline -  Crude oil futures gains 1.78% in afternoon trade 

VADER - Full , Negative News = -0.034, Neutral News = 0.77
NLTK  - Full , Negative news =  -0.917
https://www.moneycontrol.com/news/business/commodities/crude-oil-futures-gains-1-78-in-afternoon-trade-5001061.html

------
Headline -  Sensex, Nifty correct 2% intraday, end in the red; 4 factors weighing on market 

VADER - Full , Negative News = -0.003, Neutral News = 0.825
NLTK  - Full , Negative news =  -0.484
https://www.moneycontrol.com/news/business/markets/sensex-nifty-correct-2-intraday-end-in-the-red-4-factors-weighing-on-market-5000871.html

------
Headline -  SEBI internally assessing potential impact of coronavirus on markets: Whole-Time Member 

VADER - Full , Positive News = 0.071, Neutral News = 0.836
NLTK  - Full , Positive news =  5.46
https://www.moneycontrol.com/news/business/markets/sebi-internally-assessing-potential-impact-of-coronavirus-on-markets-whole-time-member-5001151.html

------
Headline 

In [6]:
import requests
response = requests.get("https://economictimes.indiatimes.com/news/politics-and-nation/union-cabinet-okays-surrogacy-regulation-bill/articleshow/74318133.cms")
soup = BeautifulSoup(response.content, "html.parser")
print(response.content)

b'<!DOCTYPE html><html xmlns:xslthelper="com.times.utils.xslthelper.XMLHelper" xmlns:xhtml="http://www.w3.org/1999/xhtml" xmlns:valurl="com.times.utilities.CMSWebUtility" xmlns:listval="com.indiatimes.cms.utilities.CMSDateUtility" xmlns:java="java" xmlns:g="http://base.google.com/ns/1.0" lang="en"><head><META http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta charset="utf-8"><title>Union Cabinet okays Surrogacy (Regulation) Bill - The Economic Times</title><link rel="image_src" href="https://img.etimg.com/thumb/msid-74318222,width-1070,height-580,imgsize-114203,overlay-economictimes/photo.jpg"><meta name="description" content="The 15 major changes suggested by the 23-member select committee of Rajya Sabha to the Surrogacy (Regulation) Bill, 2019, also included deleting the definition of &quot;infertility&quot; as the inability to conceive after five years of unprotected intercourse on the ground that it was too long a period for a couple to wait for a child."><meta name